In [5]:
import pandas as pd
import numpy as np
import math
import copy
from sklearn.metrics import *
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

<h1> Get Dataset </h1>

In [26]:
from sklearn.model_selection import train_test_split
ratings_data = pd.read_csv("./Recommendation System - Sentiment analysis/Amazon Book Reviews.csv", usecols = [0,1,5], header=0, names = ["user_id", "book_id", "rating"])
ratings_data = ratings_data.assign(newUserId = ratings_data['user_id'].astype('category').cat.codes)
ratings_data = ratings_data.assign(newBookId = ratings_data['book_id'].astype('category').cat.codes)
ratings_data.drop(["user_id","book_id"], axis=1, inplace=True)
reader = Reader(rating_scale=(1,5))
train_data = Dataset.load_from_df(ratings_data[['newUserId', 'newBookId', 'rating']], reader)
trainset = train_data.build_full_trainset()

<h1> SVD </h1>

In [77]:
from surprise import SVD
from surprise.model_selection import cross_validate
svd = SVD(n_factors=100, n_epochs=5, biased=True, random_state=15, verbose=True)
results = cross_validate(svd, train_data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0386  1.0216  1.0582  1.0395  0.0149  
MAE (testset)     0.8223  0.8073  0.8291  0.8196  0.0091  
Fit time          0.24    0.22    0.21    0.23    0.01    
Test time         0.03    0.03    0.05    0.04    0.01    


In [78]:
model=svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4


In [93]:
def estSort(pred):
    return pred.est
def recommend_books(user_id) :
    predictions = []
    for x in pd.unique(ratings_data['newBookId']):
        predictions.append(svd.predict(uid=0, iid=x))
    predictions.sort(key = estSort, reverse=True)
    recommendations = [x.iid for x in predictions[:7]]
    return recommendations

In [94]:
recommend_books(user_id=0)

[13, 106, 162, 28, 169, 142, 98]